# TEXT SUMMARIZATION USING BART TRANSFORMER MODEL

BART - Bidirectional and Auto Regressive Transformers

#### LOADING THE DATASET

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

df = load_dataset("knkarthick/dialogsum")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.65k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/442k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
df

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [ ]:
df['train'][1]['dialogue']

"#Person1#: Hello Mrs. Parker, how have you been?\n#Person2#: Hello Dr. Peters. Just fine thank you. Ricky and I are here for his vaccines.\n#Person1#: Very well. Let's see, according to his vaccination record, Ricky has received his Polio, Tetanus and Hepatitis B shots. He is 14 months old, so he is due for Hepatitis A, Chickenpox and Measles shots.\n#Person2#: What about Rubella and Mumps?\n#Person1#: Well, I can only give him these for now, and after a couple of weeks I can administer the rest.\n#Person2#: OK, great. Doctor, I think I also may need a Tetanus booster. Last time I got it was maybe fifteen years ago!\n#Person1#: We will check our records and I'll have the nurse administer and the booster as well. Now, please hold Ricky's arm tight, this may sting a little."

In [ ]:
df['train'][1]['summary']

'Mrs Parker takes Ricky for his vaccines. Dr. Peters checks the record and then gives Ricky a vaccine.'

## FINE - TUNING MODEL

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import TrainingArguments, Trainer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base")

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

'input_ids' represent the tokenized form of your input text. Each token (which could be a word or part of a word) is converted into a unique integer ID based on the model's vocabulary.

'attention_mask' is a tensor that indicates which tokens should be attended to and which should be ignored (usually padding tokens). It’s a binary mask where typically:

- 1 indicates that the token should be attended to.
- 0 indicates that the token is padding and should be ignored.

In sequence-to-sequence models, such as text summarization models, you have:

- Input IDs: Tokenized IDs of the source text (e.g., dialogue).
- Target IDs: Tokenized IDs of the target text (e.g., summary).<br>

During training, the model computes the loss between the predicted sequence and the target sequence. To ensure that padding tokens do not affect this loss calculation, padding token IDs are often replaced with -100.



In [ ]:
#tokenization

def preprocess_function(batch):
    source = batch['dialogue']
    target = batch["summary"]
    source_ids = tokenizer(source, truncation=True, padding="max_length", max_length=128)
    target_ids = tokenizer(target, truncation=True, padding="max_length", max_length=128)

    # Replace pad token id with -100 for labels to ignore padding in loss computation
    labels = target_ids["input_ids"]
    labels = [[(label if label != tokenizer.pad_token_id else -100) for label in labels_example] for labels_example in labels]

    return {
        "input_ids": source_ids["input_ids"],
        "attention_mask": source_ids["attention_mask"],
        "labels": labels
    }

df_source = df.map(preprocess_function, batched=True)

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="/content",  # Replace with your output directory
    per_device_train_batch_size=8,
    num_train_epochs=2,  # Adjust number of epochs as needed
    remove_unused_columns=False
)

In [ ]:
# Create Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=df_source["train"],
    eval_dataset=df_source["test"]
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: annisaheyy (annisaheyy-universitas-sriwijaya) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,1.886000
1000,1.740100
1500,1.687500
2000,1.488100
2500,1.442200
3000,1.425800


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=3116, training_loss=1.6065121602950874, metrics={'train_runtime': 1139.6471, 'train_samples_per_second': 21.866, 'train_steps_per_second': 2.734, 'total_flos': 1899329067417600.0, 'train_loss': 1.6065121602950874, 'epoch': 2.0})

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()

# Print evaluation results
print(eval_results)

{'eval_loss': 1.6490644216537476, 'eval_runtime': 16.8687, 'eval_samples_per_second': 88.922, 'eval_steps_per_second': 11.145, 'epoch': 2.0}


In [ ]:
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=2ded2bf80d2a14427073bbbd78e246dc26c0ea426266068440fc957ea7a2ed62
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
import evaluate

rouge = evaluate.load("rouge")

# Ambil 100 sampel untuk evaluasi (biar ringan)
# Get a sample from the 'train' split of the dataset
sample_df = df['train'].select(range(100)) # Select the first 100 rows

def generate_summary(text):
    input_text = "summarize: " + text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding="max_length").to(model.device) # Ensure inputs are on the same device as the model
    summary_ids = model.generate(inputs["input_ids"], max_length=50, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

generated_summaries = [generate_summary(text) for text in sample_df['dialogue']]
reference_summaries = sample_df['summary']

results = rouge.compute(predictions=generated_summaries, references=reference_summaries)

print("ROUGE Evaluation Results:")
for metric, score in results.items():
    print(f"{metric}: {score:.4f}")

ROUGE Evaluation Results:
rouge1: 0.4571
rouge2: 0.2412
rougeL: 0.3950
rougeLsum: 0.3951


#### SAVING THE MODEL

In [ ]:
# Save the model and tokenizer after training
model.save_pretrained("/content/your_model_directory")
tokenizer.save_pretrained("/content/your_model_directory")

('/content/your_model_directory/tokenizer_config.json',
 '/content/your_model_directory/special_tokens_map.json',
 '/content/your_model_directory/vocab.json',
 '/content/your_model_directory/merges.txt',
 '/content/your_model_directory/added_tokens.json',
 '/content/your_model_directory/tokenizer.json')

#### SUMMARIZING THE CUSTOM DATA USING SAVED MODEL AND TOKENIZER

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/your_model_directory")
model = AutoModelForSeq2SeqLM.from_pretrained("/content/your_model_directory")

# Function to summarize a blog post
def summarize(blog_post):
    # Tokenize the input blog post
    inputs = tokenizer(blog_post, max_length=1024, truncation=True, return_tensors="pt")

    # Generate the summary
    summary_ids = model.generate(inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Example blog post
blog_post = """
As Yogi Berra famously said, it’s tough to make predictions, especially about the future. But had the baseball legend spent any time observing the UN climate negotiations, he could have safely predicted that climate finance will prove to be a key sticking point at COP29 in Baku at the end of this year.

‘Who will pay and how much?’ are perennial questions at the climate talks, but this year, the discussions about climate finance will be especially prominent. At COP29, Parties to the Paris Agreement must negotiate a new climate finance goal, to replace the existing commitment from 2009 for developed countries to provide US$100 billion climate finance annually from 2020 to 2025 - a commitment that only in 2022 was starting to be fulfilled, according to a recent OECD report.

It is vital that the forthcoming Bonn Climate Change Conference sends the right political signals, and lays the procedural and technical groundwork for an ambitious climate finance deal in Baku.

A pressing need

With global warming already destabilising the climate and devastating people’s lives and livelihoods, the need for finance to reduce greenhouse gas emissions and to adapt to a warming world has never been more pressing.

The sums involved are large. The Paris Agreement’s Global Stocktake process estimates that US$5.8-5.9 trillion is required to implement Nationally Determined Contributions (NDCs) in developing countries up to 2030. They will require US$215-387 billion annually over this period for adaptation. Investments of US$1.5 trillion in renewable energy are required worldwide every year up until 2030, according to IRENA.

But these sums are also affordable and beneficial for developed countries. They should be seen in the context of ongoing investments in energy and other infrastructure: around US$2.3 trillion was invested in energy infrastructure in 2023, of which US$1.74 trillion was in clean energy. These investments will generate strong returns for their investors and reduce the costs for energy consumers.

And, crucially, they should also be seen in the context of the alternative. The latest research estimates that the world economy is already set to face a 19% income reduction within the next 26 years based on the levels of warming we have already locked in. The more we delay and the more the planet heats, the greater the economic costs will be.

Laying the foundations for a new finance goal

While financial resources are beginning to flow, they are not flowing fast enough, and certainly not flowing to those developing countries where need is greatest and access to finance is most challenging.

The UN climate framework provides mechanisms that can enable those flows of climate finance. Back in 2015, parties at the climate talks agreed to establish a “new collective quantified goal” (NCQG) for climate finance. They agreed that the NCQG would be set prior to 2025.

The  ultimate size of the NCQG will be a product of the negotiations, but Parties have agreed it must be a significant increase from the floor of US$100 billion annually. For WWF, it must be needs-based and sufficiently ambitious to meet the scale of the challenge we face, and immediately accessible to help countries that are already facing the chaos of a destabilised climate system.

While developed countries are expected to provide financial and technical support, developing countries also have a role to play. Parties are due to submit revised NDCs in 2025, presenting how they plan to reduce emissions and adapt to climate change. Developing countries have the opportunity to use their NDCs to set out how international climate finance can support them and increase their ambition. To do this, they need to know the finance will be forthcoming.
"""

# Get the summary
summary = summarize(blog_post)
print("Summary:", summary)


Summary: #1 Yogi Berra predicts that climate finance will prove to be a key sticking point at COP29 in Baku. #2 Yogi thinks it is important to establish a new climate finance goal.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/your_model_directory")
model = AutoModelForSeq2SeqLM.from_pretrained("/content/your_model_directory")

# Function to summarize a blog post
def summarize(blog_post):
    # Tokenize the input blog post
    inputs = tokenizer(blog_post, max_length=1024, truncation=True, return_tensors="pt")

    # Generate the summary
    summary_ids = model.generate(inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Example blog post
blog_post = """
World leaders, environmental activists, and scientists are ramping up their efforts to combat climate change ahead of the 29th UN Climate Change Conference (COP29), scheduled to be held in Baku, Azerbaijan, in November 2025. The summit is expected to focus on accelerating global commitments to reduce carbon emissions, transitioning to renewable energy sources, and protecting vulnerable ecosystems.

According to a new UN report released this week, global carbon emissions in 2024 slightly decreased due to increased investments in green technologies and policies promoting sustainable development. However, experts warn that the reductions are not yet enough to meet the goals of the 2015 Paris Agreement, which aims to limit global warming to 1.5 degrees Celsius above pre-industrial levels.

Countries like the United States, China, and members of the European Union have pledged additional funding and new policies to phase out coal, reduce dependency on fossil fuels, and support developing nations in their climate resilience efforts. Meanwhile, youth-led movements and environmental organizations continue to pressure governments for stronger and more immediate actions."""

# Get the summary
summary = summarize(blog_post)
print("Summary:", summary)


Summary: The 29th UN Climate Change Conference is expected to focus on accelerating global commitments to reduce carbon emissions, transitioning to renewable energy sources, and protecting vulnerable ecosystems. However, experts warn that the reductions are not yet enough to meet the goals of the 2015 Paris Agreement, which aims to limit global warming to 1.5 degrees Celsius above pre-industrial levels. The United States, China, and the European Union have pledged additional funding and new policies to phase out coal, reduce dependency on fossil fuels, and support developing nations.
